## Исследование надежности заемщиков

## 1. Ознакомление с данными

In [1]:
import pandas as pd

try:
    data = pd.read_csv('D:\docs\DS\data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2  Предобработка данных
### 2.1  Удаление пропусков
Отобразим количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

в столбцах `days_employed` и `total_income` есть пропущенные значения, начнём работу с последнего. 

на сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце следует медианным значением по каждому типу из столбца `income_type`. то есть у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### 2.2  Обработка аномальных значений
при первичном знакомстве с данными были обнаружены артефакты в столбце `days_employed`: отрицательные значения. исправим это.

In [8]:
data['days_employed'] = data['days_employed'].abs()
print(f"минимальный стаж в днях: {data['days_employed'].min()} \n"
      f"максимальный стаж в днях: {data['days_employed'].max()}"
     )

минимальный стаж в днях: 24.14163324048118 
максимальный стаж в днях: 401755.40047533


посмотрим на медианные значения стажа для каждого из типов занятости:

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

у двух типов (безработные и пенсионеры) получатся аномально большие значения. иправить такие значения сложно, поэтому оставьте их как есть. тем более этот столбец не понадобится вам для исследования.

теперь заполним пропуски:

In [15]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [16]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

проверим на наличие аномалий столбец `children`

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

избавимся от аномалий:

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data.shape

(21402, 12)

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3  Изменение типов данных
заменим вещественный тип данных в столбце `total_income`на целочисленный:

In [17]:
data['total_income'] = data['total_income'].astype(int)

### 2.4  Обработка дубликатов
обработаем неявные дубликаты в столбце `education`: здесь есть одни и те же значения, но записанные по-разному, с использованием заглавных и строчных букв. приведём их к нижнему регистру, а также выполним проверку остальных столбцов.

In [19]:
data['education'] = data['education'].str.lower()
data.duplicated().sum()

71

In [20]:
data = data.drop_duplicates()
data.shape

(21331, 12)

### 2.5 Категоризация данных

на основании диапазонов, указанных ниже, создадим `total_income_category` с категориями:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [22]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [24]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


вывдем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [25]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

напишем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [26]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [27]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## 3. Исследовательский анализ данных
### 3.1 Зависимость между количеством детей и возвратом кредита в срок

создадим сводную таблицу для категории `children`, в которой отобразим общее количество клиентов и количество должников для каждой из подкатегорий.

In [28]:
children_pivot_data = data.pivot_table(index='children', values='debt', aggfunc=['count', 'sum'])
children_pivot_data

,count,sum
,debt,debt
children,,
0,14091,1063
1,4808,444
2,2052,194
3,330,27
4,41,4
5,9,0


рассчитаем долю должников внутри каждой группы и добавим столбец в датасет:

In [30]:
children_pivot_data['debtors_percent'] = children_pivot_data['sum'] / children_pivot_data['count'] * 100

#для удобства восприятия округлим процент должников до сотых
children_pivot_data['debtors_percent'] = children_pivot_data['debtors_percent'].round(decimals=2)

# присовим столбцам более понятные имена и выведем датасет на экран:
children_pivot_data.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
children_pivot_data

,total_amount,clients_with_debt,debtors_percent
children,,,
0,14091,1063,7.54
1,4808,444,9.23
2,2052,194,9.45
3,330,27,8.18
4,41,4,9.76
5,9,0,0.00


давайте посмотрим, есть ли общие характеристики у должников без детей и у должников с двумя детьми. сравним медианный уровеь дохода и семейный статус у должников разных категорий.

In [35]:
# составим сводную таблицу, для которой из основного датасета 
# выберем должников без детей и должников с двумя детьми

compare_children = data[
    ((data['children'] == 2) & (data['debt'] == 1)) | 
    ((data['children'] == 0) & (data['debt'] == 1))
].pivot_table(
    index=['children', 'total_income_category', 'family_status'], # сведём информацию по количеству детей, категории дохода и статусу
    values=['total_income', 'debt'], # считать будем размер дохода и факт долга.
    aggfunc=['median', 'count']).reset_index() # как будем считать: медиана (для доходов) и количество должников внутри группы

compare_children.head()

children total_income_category          family_status median               \
                                                          debt total_income   
0        0                     A       гражданский брак    1.0    1030899.0   
1        0                     B  Не женат / не замужем    1.0     254793.5   
2        0                     B              в разводе    1.0     268282.0   
3        0                     B         вдовец / вдова    1.0     232026.0   
4        0                     B       гражданский брак    1.0     241915.0   

  count               
   debt total_income  
0     1            1  
1    50           50  
2    12           12  
3     7            7  
4    39           39

In [36]:
# в полученной таблице переименуем колонки для удобаства восприятия:
compare_children.columns = ['children', 'total_income_category', 'family_status', 
                            'median_debt', 'median_total_income', 'debt_count', 'income_count']

# а также выбросим из датасета столбца с медианным значением долга и подсчётом количества должников
compare_children.drop(['median_debt', 'income_count'], axis=1, inplace=True)

compare_children.head()

,children,total_income_category,family_status,median_total_income,debt_count
0,0,A,гражданский брак,1030899.0,1
1,0,B,Не женат / не замужем,254793.5,50
2,0,B,в разводе,268282.0,12
3,0,B,вдовец / вдова,232026.0,7
4,0,B,гражданский брак,241915.0,39


In [37]:
# добавим внутригрупповой процент для кажжой категории заёмщиков (без детей и с двумя детьми):
# поделим значение из столбца debt_count на общее количество должников (1063 для бездетных и 444 для тех, у кого двое)

compare_children['ingroup_percent'] = (
    compare_children.loc[compare_children['children'] == 0, 'debt_count']
    / 1063 * 100
)

compare_children.loc[
    compare_children['children'] == 2, 'ingroup_percent'
] = (
    compare_children.loc[compare_children['children'] == 2, 'debt_count']
    / 444 * 100
)

# округлим результат до сотых:
compare_children['ingroup_percent'] = compare_children['ingroup_percent'].round(decimals=2)

# отосртируем датасет по количеству детей (по возрастанию) и по уровню должников (по убыванию)
compare_children.sort_values(['children', 'debt_count'], ascending=(True, False))

,children,total_income_category,family_status,median_total_income,debt_count,ingroup_percent
10,0,C,женат / замужем,127804.0,395,37.16
9,0,C,гражданский брак,124813.0,186,17.50
6,0,C,Не женат / не замужем,135487.5,158,14.86
5,0,B,женат / замужем,252996.5,112,10.54
1,0,B,Не женат / не замужем,254793.5,50,4.70
8,0,C,вдовец / вдова,123926.5,46,4.33
7,0,C,в разводе,131091.0,43,4.05
4,0,B,гражданский брак,241915.0,39,3.67
2,0,B,в разводе,268282.0,12,1.13
3,0,B,вдовец / вдова,232026.0,7,0.66


37% должников среди заёмщиков без детей и 25% среди заёмщиков с двумя детьми имеют приблизительно одинаковые характеристики в прочих категориях:

* примерно одинаковый медианный уровень дохода клиентов категории С (127 804 и 131 068 рублей соответственно)
* одинаковый семейный статус: женат/замужем

действительно, при прочих равных условиях с увеличением количества детей в семье благонадёжность заёмщика уменьшается.

**Вывод:**  
1. клиенты без детей реже имеют задолженность по кредиту, чем семьи с детьми; чем больше детей у заёмщика, тем выше вероятность возникновения долгов по выплатам:
    * 7.54% должников среди клиентов без детей;
    * 9.23% должников среди клиентов с одним ребёнком;
    * 9.45% должников среди клиентов с двумя детьми.
2. клиентов, в семьях которых 3 и более детей, недостаточно много, чтобы сформировать репрезентативную подвыборку.

### 3.2 зависимость между семейным положением и возвратом кредита в срок
для начала посмотрим, какие типы семейного положения есть в датасете и насколько распространён каждый из них

In [38]:
print('Семейное положение', data['family_status'].value_counts(), sep='\n')

Семейное положение
family_status
женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: count, dtype: int64


прежде чем составлять сводную таблицу, проведём дополнительную категоризацию клиентов:

* так как наличие ребёнка весьма сильно влияет на возможность оплачивать кредит в срок, предположим, что одинокие люди с детьми оказываются должниками значтельно чаще, чем одиночки без детей. выделим две группы одиноких заёмщиков: с детьми и без детей;
* группы клиентов, состоящих в официальном браке и в гражданском браке, напротив, объединять не будем: посмотрим, сказывается ли отсутствие обязательств на платежёспособности клиентов.

таким образом, получаем четыре категории:
1. в браке — люди, оформившие отношение официально
2. не в браке — сожительство без вступления в брак
3. один, без детей — клиенты, не имеющие пары и не имеющие детей (в том числе разведённые или овдовевшие)
4. один, с детьми — клиенты с детьми, оставшиеся без поддержки партнёра (в том числе разведённые или овдовевшие)

напишем функцию, которая поможет нам добавить дополнительную категорию в датасет:

In [39]:
def married_or_not (row):
    status = row['family_status']
    children = row['children']
    
    if status == 'женат / замужем':
        return 'в браке'
    if status == 'гражданский брак':
        return 'не в браке'
    if status == 'Не женат / не замужем' or status == 'в разводе' or status == 'вдовец / вдова':
        if children == 0:
            return 'один, без детей'
        if children != 0:
            return 'один, с детьми'
    
data['family_grouped'] = data.apply(married_or_not, axis=1)

создадим сводную таблицу для категории `family_grouped`, в которой отобразим общее количество клиентов и количество должников для каждой из подкатегорий

In [40]:
family_pivot_data = data.pivot_table(index='family_grouped', values='debt', aggfunc=['count', 'sum'])

# сразу рассчитаем процент должников и добавим данные в сводную таблицу:
family_pivot_data['debtors_percent'] = family_pivot_data['sum'] / family_pivot_data['count'] * 100
family_pivot_data['debtors_percent'] = family_pivot_data['debtors_percent'].round(decimals=2)

# присовим столбцам более понятные имена и выведем датасет на экран:
family_pivot_data.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
family_pivot_data.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
family_grouped,,,
в браке,12261,927,7.56
"один, без детей",3893,318,8.17
не в браке,4134,385,9.31
"один, с детьми",1043,102,9.78


как и предполагалось, люди, не оформившие отношения официально, справляются с выплатой кредита хуже, чем клиенты, состоящие в браке, а самыми ненадёжными заёмщиками оказались клиенты, имеющие на иждивении несовершеннолетнего родственника. рассмотрим эту подкатегорию в разрезе прочих показателей. возможно, там есть какие-нибудь особенности, которые будут нам интересны.

In [41]:
# составим сводную таблицу, для которой из основного датасета выберем должников-одиночек с детьми:

compare_family = data[(data['family_grouped'] == 'один, с детьми') & (data['debt'] == 1)].pivot_table(
    index=['total_income_category', 'family_status'], # сведём информацию по категории дохода и статусу
    values=['total_income', 'debt'], # считать будем размер дохода и факт долга.
    aggfunc=['median', 'count']).reset_index() # как будем считать: медиана (для доходов) и количество должников внутри группы

# в полученной таблице переименуем колонки для удобаства восприятия:
compare_family.columns = ['total_income_category', 'family_status', 'median_debt', 'median_total_income', 'debt_count', 'income_count']

# а также выбросим из датасета столбца с медианным значением долга и подсчётом количества должников
compare_family.drop(['median_debt', 'income_count'], axis=1, inplace=True)

#добавим внутригрупповой процент для кажжой категории заёмщиков в зависимости от семейного статуса:
#поделим значение из столбца debt_count на общее количество должников в категории один, с детьми

compare_family['ingroup_percent'] = compare_family['debt_count'] / 102 * 100

# округлим результат до сотых:
compare_family['ingroup_percent'] = compare_family['ingroup_percent'].round(decimals=2)

# отосртируем датасет по количеству должников
compare_family.sort_values('debt_count', ascending=False)

,total_income_category,family_status,median_total_income,debt_count,ingroup_percent
3,C,Не женат / не замужем,142594.0,51,50.00
4,C,в разводе,134714.0,21,20.59
0,B,Не женат / не замужем,242307.0,12,11.76
5,C,вдовец / вдова,148167.0,8,7.84
1,B,в разводе,223195.0,6,5.88
2,B,вдовец / вдова,303546.0,2,1.96
6,D,в разводе,45402.0,2,1.96


из таблицы видно, что половина должников этой категории имеют медианный доход 142594, и этот показатель близок к тому, что мы получали ранее, проверяя наличие зависимости между количеством детей и наличием долгов по кредиту.

**Вывод:**  
* чаще всего (9.78%) не справляются со своевременным погашением кредита люди, в одиночку воспитывающие одного ребёнка или более; медианный доход таких клинетов 13 4714 рублей (разведённые) 14 2594 рублей (не женатые).
* немногим ниже процент должников среди клиентов, не оформивших отношения официально: 9.31% скорее всего, клиент берёт кредит, находясь в отношениях, а возвращает уже в одиночку, однако, в нашем датасете не хватает данных для более глубокого анализа;
* лучше всего с погашением кредита справляются семейные пары, оформившие отношения официально: всего 7.56% должников; опираясь на анализ, проведённый в п. 3.1, можно утверждать, что у большй части семейных пар, успешно возвращающих кредит, нет детей.

### 3.3 зависимость между уровнем дохода и возвратом кредита в срок
создадим сводную таблицу для категории total_income_category, в которой отобразим общее количество клиентов и количество должников для каждой из подкатегорий

In [42]:
income_category_pivot = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum'])

#сразу добавим процент должников и добавим данные в сводную таблицу:
income_category_pivot['debtors_percent'] = income_category_pivot['sum'] / income_category_pivot['count'] * 100
income_category_pivot['debtors_percent'] = income_category_pivot['debtors_percent'].round(decimals=2)

# выведем датасет на экран, предварительно переименовав колонки 
# и отсортировав в порядке возрастания доли должников в группе:
income_category_pivot.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
income_category_pivot.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
total_income_category,,,
D,349,21,6.02
B,5014,354,7.06
A,25,2,8.00
C,15921,1353,8.50
E,22,2,9.09


группы `A` и `E` слишком малочисленны, чтобы сформировать репрезентативную выборку. можно предположить, что клиенты с уровнем дохода `А` (от миллиона и выше) в принципе реже испытывают потребность в оформлении кредита, тогда как клиенты с уровнем дохода `E` (до 30 000 рублей) чаще получают отказ. однако данные текущего датасета не позволяют проверить эту информацию.

группа `D` (доход от 30 до 50 тысяч рублей) также немноголиcленна, но у неё лучший показатель, что необычно с учётом уровня дохода. сравним должников трёх категорий `B`, `C` и `D`: возможно, есть различия в целях кредита.

In [43]:
# составим сводную таблицу, для которой из основного датасета выберем должников с уровнем дохода B, C и D:

compare_income = (
    data[
        (
            (data['total_income_category'].isin(['B', 'C', 'D']))
            & (data['debt'] == 1)
        )
    ]
    .pivot_table(
        index=['total_income_category', 'purpose_category'], # сведём информацию категории дохода и цели кредита
        values=['total_income', 'debt'], # считать будем размер дохода и факт долга.
        aggfunc=['median', 'count']
    )
    .reset_index()
)

# в полученной таблице переименуем колонки для удобаства восприятия:
compare_income.columns = [
    'total_income_category', 'purpose_category', 'median_debt',
    'median_total_income', 'debt_count', 'income_count'
]

# а также выбросим из датасета столбца с медианным значением долга и подсчётом количества должников
compare_income.drop(['median_debt', 'income_count'], axis=1, inplace=True)

#добавим внутригрупповой процент для кажжой категории заёмщиков в зависимости от уровня дохода:
#поделим значение из столбца debt_count на общее количество должников в категории

compare_income['ingroup_percent'] = (
    compare_income.loc[compare_income['total_income_category'] == 'D', 'debt_count']
    / 21 * 100
)

compare_income.loc[
    compare_income['total_income_category'] == 'B', 'ingroup_percent'
] = (
    compare_income.loc[compare_income['total_income_category'] == 'B', 'debt_count']
    / 354 * 100
)

compare_income.loc[
    compare_income['total_income_category'] == 'C', 'ingroup_percent'
] = (
    compare_income.loc[compare_income['total_income_category'] == 'C', 'debt_count']
    / 1353 * 100
)

# округлим результат до сотых:
compare_income['ingroup_percent'] = compare_income['ingroup_percent'].round(decimals=2)

# отосртируем датасет по категории дохода и количеству должников
compare_income.sort_values(['total_income_category','debt_count'], ascending=(True, False))

,total_income_category,purpose_category,median_total_income,debt_count,ingroup_percent
1,B,операции с недвижимостью,251859.0,172,48.59
0,B,операции с автомобилем,257949.0,85,24.01
2,B,получение образования,258652.0,69,19.49
3,B,проведение свадьбы,263052.0,28,7.91
5,C,операции с недвижимостью,134714.0,599,44.27
4,C,операции с автомобилем,134657.0,305,22.54
6,C,получение образования,128074.0,297,21.95
7,C,проведение свадьбы,123708.0,152,11.23
8,D,операции с автомобилем,45174.0,10,47.62
9,D,операции с недвижимостью,42610.0,6,28.57


около половины должников в группах `В` и `C` имеют кредиты, связанные с недвижимостью, в то время как должники группы `D`, самой успешной, почти в половине случаев оформляли кредит на покупку автомобиля. можно предположить, что покупка автомобиля является наиболее проблемной кредитной целью &mdash; разберём это подробнее в п 3.4.

**Вывод:** 

* основная группа заёмщиков &mdash; `C`, люди с доходом 50001–200000 рублей, и они же гораздо чаще не справляются с выплатой кредита в срок.
* клиенты из категорий `D` демонстрируют наибольшую надёжность (**6.02%**), однако есть потребность проверить эти данные на более крупной выборке клиентов с данной категорией дохода.
* доверительно низкий процент должников показывает группа `В` &mdash; **7.06%** &mdash; заёмщики с уровнем дохода от двухсот тысяч до миллиона рублей.

важно отметить, что исследование выше показало, что факторами, существенно влияющими на платёжеспособность, являются семейный статус и наличие детей. рассматривать показатель платёжеспособности в отрыве от других факторов нельзя. 

### 3.4 Как разные цели кредита влияют на его возврат в срок
**создадим сводную таблицу для категории** `purpose_category`, в которой отобразим общее количество клиентов и количество должников для каждой из подкатегорий

In [44]:
purpose_pivot = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])

#сразу добавим процент должников и добавим данные в сводную таблицу:
purpose_pivot ['debtors_percent'] = purpose_pivot ['sum'] / purpose_pivot ['count'] * 100
purpose_pivot ['debtors_percent'] = purpose_pivot ['debtors_percent'].round(decimals=2)

# выведем датасет на экран, предварительно переименовав колонки 
# и отсортировав в порядке возрастания доли должников в группе:
purpose_pivot.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
purpose_pivot.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
purpose_category,,,
операции с недвижимостью,10751,780,7.26
проведение свадьбы,2313,183,7.91
получение образования,3988,369,9.25
операции с автомобилем,4279,400,9.35


ранее мы уже видели, что в отдельно взятой группе клиентов с уровнем дохода `D` почти половина проблемных заёмщиков &mdash; люди, берущие кредит для покупки авто. подсчёт количества должников по всей выборке клиентов подтверждает, что кредитная цель "операции с автомобилем" &mdash; самая проблемная.

**Вывод**  
самая многочисленная и самая надежная группа заёмщиков &mdash; люди, взявшие кредит на операции с недвижимостью (**7.26%**). основываясь на данных, полученных выше, можно сказать, что кредиты, взятые под операции с недвижимостью, чаще погашаются в срок семейными людьми, состоящими в официальных отношениях и имеющими уровень дохода 50001–200000 рублей, либо 200001–1000000 рублей.

операции с автомобилем &mdash; вторая по численности и самая ненадёжная группа заёмщиков: **9.35%** должников, почти кажлый десятый заёмщик не платит по кредиту своевременно.

прежде чем перейти к финальным выводам, проверим ещё один параметр клиента, который может оказывать влияние на платёжеспособность заёмщика: уровень образования:

In [45]:
education_category_pivot = data.pivot_table(index=['education'], values='debt', aggfunc=['count', 'sum'])

#сразу добавим процент должников и добавим данные в сводную таблицу:
education_category_pivot['debtors_percent'] = education_category_pivot['sum'] / education_category_pivot['count'] * 100
education_category_pivot['debtors_percent'] = education_category_pivot['debtors_percent'].round(decimals=2)

# выведем датасет на экран, предварительно переименовав колонки 
# и отсортировав в порядке возрастания доли должников в группе:
education_category_pivot.columns = ['total_amount', 'clients_with_debt', 'debtors_percent']
education_category_pivot.sort_values('debtors_percent')

,total_amount,clients_with_debt,debtors_percent
education,,,
ученая степень,6,0,0.00
высшее,5227,278,5.32
среднее,15075,1355,8.99
неоконченное высшее,741,68,9.18
начальное,282,31,10.99


по двум наиболее многоичсленным категориям высшее и среднее можно сделать вывод, что уровень образования заёмщика также оказывает влияние на платёжеспособность.

## 4. Общий вывод

каждый из рассмотренных параметров клиентов влияет на платёжеспособность:

* **дети**. клиенты, не имеющие детей, реже имеют задолженность по кредиту, чем клиенты с 2 детьми (**7.54%** против **9.45%**). 
* **семейное положение**. наименьшая доля должников (**7.56%**) среди клиентов, состоящих в официальном браке, наибольшая (**9.78%**) &mdash; среди одиноких людей с ребёнком
* **доход**. с увеличением уровня дохода снижается риск задолженности по выплате кредита:
    * 7.06% должников среди клиентов группы `В` (доход от 200 тысяч до 1 миллиона);
    * 8.50% должников среди группы `С` (доход от 50 до 200 тысяч).
    однако, выборка `В` в три раза меньше, чем выборка `С`. для более глубокого анализа следует выровнять количество наблюдений в обеих выборках, желательно не уменьшать выборку `С`, а собрать больше данных по клиентам группы `B`.
* **цели**. наибольшая доля должников среди тех, кто оформил кредит на приобретение машины (**9.35%**); операции с недвижимостью, напротив, именют наименьшую долю должников (**7.26%**), несмотря на б*о*льшую стоимость кредита.
* **образование**. также косвенно влияет на платёжеспособность клиента: люди с высшим образованием берут кредит реже и реже имеют задолженность, чем люди со средним образованием (**5.32%** против **8.99%**)

**портрет наиболее надёжного заёмщика**:  
клиент, состоящий в законном браке, без детей, имеющий высшее образование и уровень дохода от 200 тысяч до 1 миллиона рублей.

**портрет наименее надёжного заёмщика**:  
клиент, не состоящий в отношениях, но имеющий 1-2 детей, имеющий среднее образование и уровень дохода от 50 до 200 тысяч рублей.

однако следует отметить, что для большей точности анализа все перечисленные выше характеристики следует рассмотреть во взаимосвязи и изучить их корреляцию. например: как уровень образования, должность и стаж клиента влияют на его доход. такой анализ поможет нам выявить наиболее надёжных клиентов.

также было бы полезно рассмотреть:
* корреляцию между уровнем дохода, наличием долгов и суммой кредитования
* корреляцию между доходом клиента, общим доходом семьи и наличием долгов по выплате